<table>
    <tr><td> <b>Group</b> </td><td>Your group number</td></tr>
    <tr><td> <b>Names</b> </td><td>Your names</td></tr>
</table>

In [2]:
#%matplotlib inline
import numpy as np
import operator
import pylab

pylab.rcParams['figure.figsize'] = (10.0, 8.0)


# Decision Trees

In this lab, we will implement the C4.5 algorithm for classification with decision trees. Please make sure that you read the complete description and code below.


## Data

Let's start by loading the datasets that we will be working with. The ```mushroom``` dataset is a dataset describing the features of different species of mushrooms, and the goal is to derive simple rules to predict whether a species is poisonous or not.

In [5]:
# This code is given
data = np.load("uci-mushrooms.npz")
# data = np.load("hiv.npz")
print(data.keys())

d = data['inputs']
l = data['targets']
fname = data['feature_names']
print(fname)

['inputs', 'targets', 'feature_names']
[b'cap-shape' b'cap-surface' b'cap-colour' b'bruises?' b'odour'
 b'gill-attachment' b'gill-spacing' b'gill-size' b'gill-colour'
 b'stalk-shape' b'stalk-root' b'stalk-surface-above-ring'
 b'stalk-surface-below-ring' b'stalk-colour-above-ring'
 b'stalk colour-below-ring' b'veil type' b'veil colour' b'Number-of-rings'
 b'Type-of-ring' b'spore-print-colour' b'population' b'habitat']


## Helper functions

Here are a few functions to deal with categorical labels

In [9]:
# This code is given

def listValues(inputlist):
    """List the possible values of a given feature in a dataset"""
    values = set([])
    for x in inputlist:
        values.add(x)
    return list(values)

def countLabels(labelList):
    """Count how often each individual label is repeated in the list"""
    counts = {}
    for x in labelList:
        if x not in counts:
            counts[x] = 1
        else:
            counts[x] += 1
    return counts

def majorityLabel(labelList):
    """Return the most common label in a list"""    
    counts = countLabels(labelList)
#    print()"Majority label:", counts)
    return max(counts.items(), key=operator.itemgetter(1))[0]


print(listValues(l))
print(listValues(d[:,0]))
print(countLabels(l))
print(majorityLabel(d[:,1]))

[b'EDIBLE', b'POISONOUS']
[b'KNOBBED', b'SUNKEN', b'CONVEX', b'CONICAL', b'BELL', b'FLAT']
{b'EDIBLE': 4488, b'POISONOUS': 3928}
b'SCALY'


## Computing the entropy of a list of categorical values

The decision tree makes its decision based on decision nodes, and these nodes are evaluated based on information gain they provide on the training data. Information gain is computed as 
$$
IG = H(parent) - \sum_j \frac{N_j}{N} H(child=j)
$$
the entropy of the data associated with the parent, minus the weighted entropy of the data-subsets associated with the chidlren

**question 1 [5 credits]]** Implement a helper function that computes the entropy of a set of categorical labels


In [11]:
# Answer to Question 1
#ANSQ1
def entropy(labelList):
    # implement entropy computation here
    return 0.0
        
print(entropy(l))
               
# Comment: it is possible for the entropy to be larger than one, if you 
# consider distributions over more than 2 values. Don't panic :-)
#/ANSQ1

0.9968038285222955


## Implementing the tree

Below, you are given the code to make a decision tree, train it and classify datapoints using binary decision nodes based on categorical variables. 

The implementation creates ```Node``` objects, which provide three methods: ```classify```, which returns the predicted label for a datapoint, and ```entropy``` and ```informationGain```, which are used in training. How ```classify``` does its work depends on the type of node, and is implemented in the derived classes. 

```Leaf``` nodes return a fixed classification result that does not depend on the datapoint at all: the classification is defined by the path that was traveled through the tree to reach that node (and, of course, that path depends on the value of the datapoint). ```Leaf``` nodes simply return the majority label of the training data that is associated with them. All other nodes allow have children, and their ```classify``` method consists of deciding which child to select to classify the datapoint. A first implementation of such a node is the ```BinaryNode```, which makes a binary decision on which of its two children should be responsible for the further classification.

Each ```Node```'s constructor is given training data which is used to optimise the ```Node```'s classification strategy. ```Leaf``` nodes use this data to select the majority class, ```BinaryNodes``` select the most informative feature and that feature's most informative value in their data. When ```classify```ing a new datapoint, they select which child should perform the classification based on whether or not the corresponding feature of the datapoint equals that value, or not.

In [12]:
# This code is given

class Node:
    """Abstract class for a decision tree node"""
    def classify(self, datapoint):
        """Return the classification label for the given datapoint"""
        raise Exception("Not Implemented")        
    def entropy(self):
        """Return the entropy of the training data associated with this node"""
    def informationGain(self):
        """Return the information gain of this node on the training data """
        raise Exception("Not Implemented by derived class")

class Leaf(Node):
    """Leaf node, i.e., a node with no children"""
    def __init__(self,inputs, targets):
#        print "Inputs, targets:", inputs.shape, targets.shape
        
        self.v = majorityLabel(targets)
        self.inputs = inputs
        self.targets = targets
        self.children = None
        self.H = entropy(targets)
        
    def entropy(self):
        """Return the entropy of the training data associated with this node"""
        return self.H
    
    def informationGain(self):
        """No splitting of the data, so no information gain"""
        return 0.
    
    def classify(self,datapoint):
        """Return the majority class for this node (and ignore the actual value of the datapoint)"""
        return self.v
    
    def __str__(self): 
        """Used for printing trees"""
        return "\t\t\t=> %s [Leaf]" % (self.v)
    
class BinaryNode(Node):
    """Make a decision based on whether a given feature equals a specific value, or not"""
    def __init__(self, inputs, targets, feature):
        self.feature = feature
        self.value   = None
        self.IG      = 0.0
        self.H       = entropy(targets)                     # Entropy of unsplit data
        self.inputs  = inputs
        self.targets = targets
        N = float(targets.size)                             # Total number of datapoints

        for value in listValues(inputs[:,feature]):         # Go through all possible values for the given feature
            indicesTrue  = inputs[:,feature]==value         # Split the training data based on that feature/value
            indicesFalse = inputs[:,feature]!=value

            if indicesTrue.all() or indicesFalse.all():     # If none (or all) datapoints have the same value for that feature                  
                continue                                    # ... keep going

            children = [ Leaf(inputs[indicesTrue,:], targets[indicesTrue]),    # Temporarily create children
                         Leaf(inputs[indicesFalse,:], targets[indicesFalse]) ] # based on this split
            conditions = [ "%s==%s" % (fname[feature], value), "%s!=%s" % (fname[feature], value) ] # (for printing)

            Nt = float(children[0].targets.size)        # N.o. datapoints in True condition
            Nf = float(children[1].targets.size)        # N.o. datapoints in False condition
            pt = Nt/N
            pf = Nf/N
            ig = self.H - pt * children[0].entropy() - pf * children[1].entropy() # Compute the information gain 
                                                        # with the temporary children
            if ig > self.IG:                            # Check whether that information gain was better than what we had
                self.value = value                      # and if so, update the node and make those children permanent 
                self.IG = ig
                self.children = children
                self.conditions = conditions
    
    def entropy(self):
        return self.H
    
    def informationGain(self):
        """Return the entropy gain of splitting the given data according to 
        the specified feature and value"""
        return self.IG
    
    def classify(self, x):
        if x[self.feature] == self.value:
            return self.children[0].classify(x)
        else:
            return self.children[1].classify(x)
        
    def __str__(self):
        return " => Binary decision: '%s =?= %s'" % (fname[self.feature], self.value )
    
    

In [13]:
# This code implements the C4.5 training algorithm for decision trees.
# It is given BUT MUST BE MODIFIED! (Question 2)

# Don't worry too much about the following line, it'll come in handy later when 
# we work with heterogeneous features 
allBinary = [ BinaryNode for i in range(d[0].size) ] # all features require Binary Nodes

def c45(inputs, targets, featureType = allBinary, minSize=0):
    """Implement the C4.5 training algorithm for a classification tree"""
    ig = 0.      # Initialise the information gain as 0
    res = None   # Keep track of the best feature so far
    for feature in range(inputs.shape[1]): # Cycle through all possible features
        n = featureType[feature](inputs,targets,feature) # Create a Node of the right type for that feature
        if n.informationGain() > ig: # If that node is better than what we'd found until now, 
            ig = n.informationGain() # keep that node
            res = n
    if not res: # IF no interesting feature was found for this data, return None
#        print "WARNING: No informative feature found" # Comment this out for more verbose output
        return res
    for i in range(len(res.children)): # Now go through the children of the best node we've created
        if res.children[i].entropy() > 0.: # If that child can still be split up
            c = c45(res.children[i].inputs, res.children[i].targets, featureType, minSize) # recursively try to split it up
            if c: # if splitting it up was successful (return value is not None)
                res.children[i]=c # update the child with the created subtree
    return res


root = c45(d,l)

In [15]:
# This code is given

def prettyPrint(tree, cond="", dlist = [True]):
    indent = ""
    for d in dlist[:-1]:
        indent += "   " if d else "|  "
    print(indent + "+-> " + cond + " " + tree.__str__())
    if tree.children:
        for cond,cld in zip(tree.conditions[:-1],tree.children[:-1]):
            prettyPrint(cld,cond,dlist + [ False ])
        prettyPrint(tree.children[-1], tree.conditions[-1], dlist+[ True ])

    

prettyPrint(root)


+->   => Binary decision: 'b'odour' =?= b'NONE''
   +-> b'odour'==b'NONE'  => Binary decision: 'b'spore-print-colour' =?= b'GREEN''
   |  +-> b'spore-print-colour'==b'GREEN' 			=> b'POISONOUS' [Leaf]
   |  +-> b'spore-print-colour'!=b'GREEN'  => Binary decision: 'b'stalk-surface-below-ring' =?= b'SCALY''
   |     +-> b'stalk-surface-below-ring'==b'SCALY'  => Binary decision: 'b'gill-size' =?= b'BROAD''
   |     |  +-> b'gill-size'==b'BROAD' 			=> b'EDIBLE' [Leaf]
   |     |  +-> b'gill-size'!=b'BROAD' 			=> b'POISONOUS' [Leaf]
   |     +-> b'stalk-surface-below-ring'!=b'SCALY'  => Binary decision: 'b'cap-surface' =?= b'GROOVES''
   |        +-> b'cap-surface'==b'GROOVES' 			=> b'POISONOUS' [Leaf]
   |        +-> b'cap-surface'!=b'GROOVES'  => Binary decision: 'b'gill-size' =?= b'BROAD''
   |           +-> b'gill-size'==b'BROAD' 			=> b'EDIBLE' [Leaf]
   |           +-> b'gill-size'!=b'BROAD'  => Binary decision: 'b'bruises?' =?= b'BRUISES''
   |              +-> b'bruises?'==b'BRUISES'

# Training 

The code above trains a decision tree.

**Question 2 [5 credits]** Modify the training function provided above to stop splitting nodes when the corresponding training data contains less than ```minSize``` datapoints, where ```minSize``` is a parameter to the function. Split the data into 80\% train and 20\% test, and use 5-fold cross-validation on the training set to find a good value for this parameter (You can use external libraries to split up the data if that's easier). Then report the confusion matrix, precision, recall and accuracy that you obtain on the test set.


In [18]:
# Answer to Q2
#ANSQ2
data = np.load("hiv.npz")
d = data["inputs"]
l = data["targets"]
fname = [ "feature-%d" % x for x in range(d.shape[1]) ]

def c45(inputs, targets, featureType = allBinary, minSize=0):
    #Copy-paste the previous code for c45 here, and modify it.


#/ANSQ2

### Continuous values

Until now, we have only dealt with categorical features. In the next part of the lab, you will deal with mixed categorical and continuous features.

One way to deal with continuous features is to split the dataset based on whether the feature is larger than a certain threshold, or not. The number of possible threshold values to consider is, of course, infinite in theory, but when deciding what the threshold value should be, based on a training dataset, only threshold values that will result in a different classification of those training examples matter: those are values that are smaller than one datapoint's feature value and larger than another's. Moreover, when considering a threshold in between two feature values, any value of the threshold in that range will result in the same classification, but to maximise generalisation it makes sense to maximise the margin: that is, to take a threshold value that is in the middle between the two.

Also note that it only makes sense to consider ranges of feature values that are disjoint, i.e., consider thresholds that are in the middle between neighbouring datapoints (along that feature's dimension), not in the middle between any pair of datapoints.

**Question 3 [10 credits]**. You guessed it, implement a node ```ContNode``` that selects a threshold value based on its training subset and classifies new datapoints based on whether they are smaller than that value, or not.

In [20]:
# This code must be modified.
#ANSQ3
class ContNode(Node):
    """Make a binary decision based on a threshold, for the value of the given feature"""
    def __init__(self, inputs, targets, feature):
        self.feature = feature
        self.IG      = 0.0
        self.H       = entropy(targets)                     # Entropy of unsplit data
        self.inputs  = inputs
        self.targets = targets

        # Your code must fill in self.value here
        
    def entropy(self):
        return self.H
    
    def informationGain(self):
        """Return the entropy gain of splitting the given data according to 
        the specified feature and value"""
        return self.IG
    
    def classify(self, x):
        if x[self.feature] >= self.value:
            return self.children[0].classify(x)
        else:
            return self.children[1].classify(x)
        
    def __str__(self):
        return "=> threshold decision: %s >= %f" % (fname[self.feature],self.value)
#/ANSQ3

## Classification based on mixed features

The following dataset contains 14 features describing people's situation, and the classification task is to predict whether they earn more than \$50k a year or not. The categorical decision nodes we implemented above can work with any representation of categorical data, but the continuous decision node requires a numerical representation of the feature. Numpy arrays cannot easily contain both strings and numbers, and so to simplify our life, we converted the dataset to all numeric inputs and string targets.

To allow our classifier to work with mixed features, we use a numerical representation of the categorical features in the following. So, let's first load the data:

In [27]:
# This code is given

income = np.load('income.npz')
print(income.keys())
inputs = income['inputs']
targets = income['targets']
fname = income['fnames']
ftypes = income['ftypes']

print(ftypes)

['inputs', 'catindices', 'fnames', 'targets', 'ftypes']
[b'continuous' b'categorical' b'continuous' b'categorical' b'continuous'
 b'categorical' b'categorical' b'categorical' b'categorical'
 b'categorical' b'continuous' b'continuous' b'continuous' b'categorical']


**Question 4 [10 credits]** Take 1000 training datapoints at random from the dataset, and another 1000 test points that are distinct from those (don't mix train and test). Train a decision tree on the train set and report the confusion matrix obtained on the test set. What are the obtained precision, recall and accuracy? 

In [24]:
# Answer to Q4
#ANSQ4

#/ANSQ4

+->   => Binary decision: 'b'marital-status' =?= 1.0'
   +-> b'marital-status'==1.0  => Binary decision: 'b'capital-gain' =?= 0.0'
   |  +-> b'capital-gain'==0.0  => Binary decision: 'b'occupation' =?= 1.0'
   |  |  +-> b'occupation'==1.0  => Binary decision: 'b'education' =?= 3.0'
   |  |  |  +-> b'education'==3.0 			=> b'>50K' [Leaf]
   |  |  |  +-> b'education'!=3.0  => Binary decision: 'b'education' =?= 0.0'
   |  |  |     +-> b'education'==0.0  => Binary decision: 'b'capital-loss' =?= 0.0'
   |  |  |     |  +-> b'capital-loss'==0.0  => Binary decision: 'b'relationship' =?= 3.0'
   |  |  |     |  |  +-> b'relationship'==3.0 			=> b'<=50K' [Leaf]
   |  |  |     |  |  +-> b'relationship'!=3.0  => Binary decision: 'b'hours-per-week' =?= 70.0'
   |  |  |     |  |     +-> b'hours-per-week'==70.0 			=> b'<=50K' [Leaf]
   |  |  |     |  |     +-> b'hours-per-week'!=70.0  => Binary decision: 'b'hours-per-week' =?= 55.0'
   |  |  |     |  |        +-> b'hours-per-week'==55.0 			=> b'>50K' [